# Libraries

In [90]:
import numpy as np
import pandas as pd
import itertools
import random

from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# For reproducability.
seed = 2003
random.seed(seed)
np.random.seed(seed)

# Import Datasets

In [91]:
assessments = pd.read_csv(r'..\..\anonymisedData\assessments.csv')
courses = pd.read_csv(r'..\..\anonymisedData\courses.csv')
studentAssessment = pd.read_csv(r'..\..\anonymisedData\studentAssessment.csv')
studentInfo = pd.read_csv(r'..\..\anonymisedData\studentInfo.csv')
studentRegistration = pd.read_csv(r'..\..\anonymisedData\studentRegistration.csv')
studentVle = pd.read_csv(r'..\..\anonymisedData\studentVle.csv')
vle = pd.read_csv(r'..\..\anonymisedData\vle.csv')

# Generate Dropout Data Based On Module Length

In [92]:
# Get module timestamps.
module_timestamps = courses.copy()
module_timestamps['timestamp'] = module_timestamps['module_presentation_length'] # Set the timestamp here.
print(module_timestamps)

   code_module code_presentation  module_presentation_length  timestamp
0          AAA             2013J                         268        268
1          AAA             2014J                         269        269
2          BBB             2013J                         268        268
3          BBB             2014J                         262        262
4          BBB             2013B                         240        240
5          BBB             2014B                         234        234
6          CCC             2014J                         269        269
7          CCC             2014B                         241        241
8          DDD             2013J                         261        261
9          DDD             2014J                         262        262
10         DDD             2013B                         240        240
11         DDD             2014B                         241        241
12         EEE             2013J                         268    

## VLE Tables

In [93]:
# Merge with an inner join.
vle_df = pd.merge(studentVle, vle, on=['code_module', 'code_presentation', 'id_site'], how='inner')
vle_df.drop(columns=['week_from', 'week_to'], inplace=True) # Drop columns.

# Filter VLE and assessments.
# Filter rows where date is <= timestamp
vle_df_custom = vle_df.merge(module_timestamps[['code_module', 'code_presentation', 'timestamp']], on=['code_module', 'code_presentation'], how='left')
vle_df_custom = vle_df_custom[vle_df_custom['date'] <= vle_df_custom['timestamp']]

# Aggregate total clicks and active days.
vle_agg_custom = vle_df_custom.groupby(['code_module', 'code_presentation', 'id_student']).agg(
    total_clicks=('sum_click', 'sum'),
    num_days_active=('date', 'nunique'),
).reset_index()

# Add timestamp again.
vle_agg_custom = vle_agg_custom.merge(
    module_timestamps[['code_module', 'code_presentation', 'timestamp']], 
    on=['code_module', 'code_presentation'], 
    how='left'
)

# Normalize active days.
vle_agg_custom['days_active_norm'] = vle_agg_custom['num_days_active'] / vle_agg_custom['timestamp']
vle_agg_custom['days_active_norm'] = round(vle_agg_custom['days_active_norm'].clip(upper=1), 2)
vle_agg_custom = vle_agg_custom.drop(columns=['timestamp', 'num_days_active'])

# Get full list of students by module/presentation.
student_keys = studentInfo[['code_module', 'code_presentation', 'id_student']].drop_duplicates()

# Merge with all students to include those with 0 VLE.
vle_agg_custom = student_keys.merge(vle_agg_custom, on=['code_module', 'code_presentation', 'id_student'], how='left')

# Fill NaNs for students with no activity.
vle_agg_custom['total_clicks'] = vle_agg_custom['total_clicks'].fillna(0)
vle_agg_custom['days_active_norm'] = vle_agg_custom['days_active_norm'].fillna(0)

# Build the activity click matrix.
vle_activity_ohe_custom = pd.pivot_table(
    vle_df_custom,
    index = ['code_module', 'code_presentation', 'id_student'],
    columns = 'activity_type',
    values = 'sum_click',
    aggfunc = 'sum',
    fill_value = 0
).reset_index()

# Make sure students with no activity are included.
vle_activity_ohe_custom = student_keys.merge(
    vle_activity_ohe_custom,
    on=['code_module', 'code_presentation', 'id_student'],
    how='left'
)

# Fill missing click values with 0.
activity_cols = [col for col in vle_activity_ohe_custom.columns if col not in ['code_module', 'code_presentation', 'id_student']]
vle_activity_ohe_custom[activity_cols] = vle_activity_ohe_custom[activity_cols].fillna(0)

# One-hot encode clicks by activity type.
vle_unique_module_activity_custom = (
    vle_df_custom[['code_module', 'activity_type']]
    .drop_duplicates()
    .assign(is_available=1)
    .pivot_table(index='code_module', columns='activity_type', values='is_available', fill_value=0)
).reset_index()

vle_unique_module_activity_custom.columns = ['code_module'] + [f"has_{col}" for col in vle_unique_module_activity_custom.columns[1:]]

# Merge vle_unique_module_activity_custom with vle_activity_ohe_custom.
vle_activity_features_custom = pd.merge(
    vle_activity_ohe_custom,
    vle_unique_module_activity_custom,
    on='code_module',
    how='left'
)

# Merge with the previously aggregated VLE dfs.
vle_df_custom = pd.merge(
    vle_agg_custom,  # total_clicks and num_days_active_norm.
    vle_activity_features_custom,
    on=['code_module', 'code_presentation', 'id_student'],
    how='left'
)

## Student Tables

In [94]:
student_reg_and_courses_df = pd.merge(studentRegistration, courses, on=['code_module', 'code_presentation'], how='inner')
student_details_df = pd.merge(student_reg_and_courses_df, studentInfo, on=['code_module', 'code_presentation', 'id_student'], how='inner')

## Assessment Tables

In [95]:
# Impute the missing exam dates with the module_presentation_length date.
assessments.loc[assessments['date'].isna(), 'date'] = assessments.loc[assessments['date'].isna()].merge(
    courses[['code_module', 'code_presentation', 'module_presentation_length']],
    on=['code_module', 'code_presentation'],
    how='left'
)['module_presentation_length'].values

assessment_df = pd.merge(assessments, studentAssessment, on=['id_assessment'], how='inner')

# Rearrange column names.
assessment_df = assessment_df[['id_student', 'code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'date_submitted', 'weight', 'is_banked', 'score']]
assessment_df = assessment_df.rename(columns={'date': 'date_due'}) # Rename 'date' to 'date_due'

# Filter rows where date_due is <= timestamp.
assessment_df_custom = assessment_df.merge(module_timestamps[['code_module', 'code_presentation', 'timestamp']], on=['code_module', 'code_presentation'], how='left')
assessment_df_custom = assessment_df_custom[assessment_df_custom['date_due'] <= assessment_df_custom['timestamp']]
assessment_results_df_custom = assessment_df_custom.copy() # Make a copy of dataset.

## Weighted Score ##
# For each assessment due before timestamp.
assessment_due = assessment_df_custom[['code_module', 'code_presentation', 'id_assessment', 'weight', 'assessment_type', 'date_due']].drop_duplicates()

# Cross join students and assessments by module/presentation.
students_assessments = student_keys.merge(assessment_due, on=['code_module', 'code_presentation'], how='left')

# Merge with actual student submissions.
students_assessments = students_assessments.merge(
    studentAssessment[['id_student', 'id_assessment', 'date_submitted', 'is_banked', 'score']],
    on=['id_student', 'id_assessment'],
    how='left'
)

# Fill missing scores with 0 (for not submitted).
students_assessments['score'] = students_assessments['score'].fillna(0)

# Compute total weight per student/module (based on all assessments that were due)
total_weight_df = students_assessments.groupby(['id_student', 'code_module', 'code_presentation'])['weight'].sum().reset_index(name='total_weight')

# Merge total weight back.
students_assessments = students_assessments.merge(total_weight_df, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Compute weighted score per assessment.
students_assessments['weighted_score_component'] = (
    (students_assessments['weight'] / students_assessments['total_weight']) *
    students_assessments['score']
)

# Final weighted score per student/module.
assessment_results_df_custom = students_assessments.groupby(['id_student', 'code_module', 'code_presentation'])['weighted_score_component'].sum().reset_index(name='weighted_score')
assessment_results_df_custom['weighted_score'] = assessment_results_df_custom['weighted_score'].round(2)

## Banked Rate ##
# Count total assessments per student per module presentation.
total_assessments_custom = assessment_df_custom.groupby(['id_student', 'code_module', 'code_presentation'])['is_banked'].count().reset_index()
total_assessments_custom = total_assessments_custom.rename(columns={'is_banked': 'total_assessments_custom'})

# Count number of banked assessments (is_banked == 1).
banked_assessments = assessment_df_custom[assessment_df_custom['is_banked'] == 1].groupby(
    ['id_student', 'code_module', 'code_presentation'])['is_banked'].count().reset_index()
banked_assessments = banked_assessments.rename(columns={'is_banked': 'banked_assessments'})

# Merge the two.
banked_rate_df_custom = total_assessments_custom.merge(
    banked_assessments, 
    on=['id_student', 'code_module', 'code_presentation'], 
    how='left'
)

# Fill NaN banked counts with 0 and compute ratio.
banked_rate_df_custom['banked_assessments'] = banked_rate_df_custom['banked_assessments'].fillna(0)
banked_rate_df_custom['banked_rate'] = round(banked_rate_df_custom['banked_assessments'] / banked_rate_df_custom['total_assessments_custom'], 2)

# Drop helper columns.
banked_rate_df_custom = banked_rate_df_custom.drop(columns=['total_assessments_custom', 'banked_assessments'])

# Merge the banked_rate into assessment_results_df_custom.
assessment_results_df_custom = assessment_results_df_custom.merge(
    banked_rate_df_custom,
    on=['id_student', 'code_module', 'code_presentation'],
    how='left'
)

## Late Submission Rate ##
late_submission_df_custom = assessment_df_custom.copy()
late_submission_df_custom[(late_submission_df_custom['assessment_type'] == 'Exam') & (late_submission_df_custom['date_submitted'] > late_submission_df_custom['date_due'])]

# Calculate late submission: submission is late if date_submitted > date_due, and date_due is not missing.
late_submission_df_custom['late_submission'] = ((late_submission_df_custom['date_submitted'] > late_submission_df_custom['date_due']) & late_submission_df_custom['date_due'].notna())

# Group to get total late submissions per student/module.
late_counts = late_submission_df_custom.groupby(['id_student', 'code_module', 'code_presentation'])['late_submission'].sum().reset_index(name='late_count')

# Get total assessments submitted per student/module.
total_per_student_submission_counts_custom = late_submission_df_custom.groupby(['id_student', 'code_module', 'code_presentation']).size().reset_index(name='total_submitted')
late_rate_df_custom = pd.merge(late_counts, total_per_student_submission_counts_custom, on=['id_student', 'code_module', 'code_presentation']) # Merge the late counts and total counts dfs.
late_rate_df_custom['late_rate'] = round(late_rate_df_custom['late_count'] / late_rate_df_custom['total_submitted'], 2) # Calculate late submission rate and round it to 2 decimal places.
late_rate_df_custom.drop(columns=['late_count', 'total_submitted'], inplace=True) # Drop helper columns.

## Fail Rate ##
students_in_module_custom = assessment_df_custom[['id_student', 'code_module', 'code_presentation']].drop_duplicates()
all_assessments_custom = assessment_df_custom[['id_assessment', 'code_module', 'code_presentation']].drop_duplicates()
rows = []

# Iterate over each module/presentation group separately.
for (module, presentation), students_grp in students_in_module_custom.groupby(['code_module', 'code_presentation']):
    assessments_grp = all_assessments_custom[
        (all_assessments_custom['code_module'] == module) & 
        (all_assessments_custom['code_presentation'] == presentation)
    ]
    for student, assessment in itertools.product(students_grp['id_student'], assessments_grp['id_assessment']):
        rows.append({
            'code_module': module,
            'code_presentation': presentation,
            'id_student': student,
            'id_assessment': assessment
        })

cartesian_df_custom = pd.DataFrame(rows)

# Merge the full student assessment pairs with actual scores (NaN means not submitted).
merged_student_assessment_pairs = cartesian_df_custom.merge(
    assessment_df_custom[['id_student', 'id_assessment', 'score']],
    on=['id_student', 'id_assessment'],
    how='left'
)

# Mark fail if score < 40 or score is missing (NaN).
merged_student_assessment_pairs['fail'] = merged_student_assessment_pairs['score'].isna() | (merged_student_assessment_pairs['score'] < 40)

# Calculate fail counts and total assessments per student/module.
fail_rate_df_custom = merged_student_assessment_pairs.groupby(['id_student', 'code_module', 'code_presentation']).agg(
    total_assessments=('fail', 'count'),
    total_fails=('fail', 'sum')
).reset_index()

# Calculate fail rate.
fail_rate_df_custom['fail_rate'] = (fail_rate_df_custom['total_fails'] / fail_rate_df_custom['total_assessments']).round(2)
fail_rate_df_custom = fail_rate_df_custom.drop(columns=['total_assessments', 'total_fails'])

## Merge Assessment Tables ##
student_assessments_df_custom = pd.merge(assessment_results_df_custom, late_rate_df_custom, on=['id_student', 'code_module', 'code_presentation'], how='inner')
student_assessments_df_custom = pd.merge(student_assessments_df_custom, fail_rate_df_custom, on=['id_student', 'code_module', 'code_presentation'], how='inner')

In [96]:
assessment_results_df_custom

,id_student,code_module,code_presentation,weighted_score,banked_rate
0,3733,DDD,2013J,0.00,NaN
1,6516,AAA,2014J,63.50,0.0
2,8462,DDD,2013J,17.45,0.0
3,8462,DDD,2014J,21.50,1.0
4,11391,AAA,2013J,82.40,0.0
...,...,...,...,...,...
32588,2702660,FFF,2014J,0.00,NaN
32589,2707979,DDD,2013B,0.00,NaN
32590,2710343,DDD,2013B,0.00,NaN
32591,2710343,DDD,2014B,0.00,NaN


## Final Merge

In [97]:
student_df_custom = pd.merge(student_details_df, vle_df_custom, on=['id_student', 'code_module', 'code_presentation'], how='left')
student_df_custom = pd.merge(student_df_custom, student_assessments_df_custom, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Preprocess Dataset

In [98]:
## Date Registration ##
median_reg_date = student_df_custom.date_registration.median() # Get median registration date.
# Replace null values with date unregistration to the median date.
student_df_custom['date_registration'] = np.where((student_df_custom['date_registration'].isnull()), 
                                                    student_df_custom['date_unregistration'] + median_reg_date,
                                                    student_df_custom['date_registration'])

# Replace remaining null values with -57 (with null date_unregistration).
student_df_custom['date_registration'] = np.where( (student_df_custom['date_registration'].isnull()), median_reg_date, student_df_custom['date_registration'])

## Gender and Disability ##
student_df_custom['gender'] = student_df_custom['gender'].map({'M': 1, 'F': 0}) # Convert gender: M = 1, F = 0.
student_df_custom['disability'] = student_df_custom['disability'].map({'Y': 1, 'N': 0}) # Convert disability: Y = 1, N = 0.

## Highest Education ##
he_encoding = {
    'No Formal quals': 0,
    'Lower Than A Level': 1,
    'A Level or Equivalent': 2,
    'HE Qualification': 3,
    'Post Graduate Qualification': 4
}
student_df_custom['highest_education'] = student_df_custom['highest_education'].map(he_encoding).astype(int)

## Age Band ##
age_mapping = {
    '0-35': 0,
    '35-55': 1,
    '55<=': 2
}
student_df_custom['age_band'] = student_df_custom['age_band'].map(age_mapping).astype(int)

## Final Result ##
final_result_mapping = {
    'Distinction': 3,
    'Pass': 2,
    'Fail': 1,
    'Withdrawn': 0
}
student_df_custom['final_result'] = student_df_custom['final_result'].map(final_result_mapping).astype(int)

## Total Clicks ##
# Calculate mean 'total_clicks' for students who passed and have non-null `total_clicks`.
mean_total_clicks_for_pass_custom = int(student_df_custom[(student_df_custom['final_result'] == 2) & (student_df_custom['total_clicks'].notna())]['total_clicks'].mean())

# Impute missing `total_clicks` for students who have passed.
student_df_custom.loc[(student_df_custom['final_result'] == 2) & (student_df_custom['total_clicks'].isna()), 'total_clicks'] = mean_total_clicks_for_pass_custom

## Banked Rate ##
student_df_custom['banked_rate'] = student_df_custom['banked_rate'].fillna(0)

## Weighted Score ##
# Calculate and store the mean pass score.
mean_pass_score_custom = round(student_df_custom[(student_df_custom['final_result'] == 2) & (student_df_custom['weighted_score'].notna())]['weighted_score'].mean(), 2)

# Impute the mean pass score to the three students who have passed but have null weighted scores.
student_df_custom.loc[(student_df_custom['final_result'] == 2) & (student_df_custom['weighted_score'].isna()),'weighted_score'] = round(mean_pass_score_custom, 2)

# For the Failed/Withdrawn students, the weighted score will be set to 0.
student_df_custom['weighted_score'] = student_df_custom['weighted_score'].fillna(0)

## Late and Fail Rate ##
### Late Rate ###
# Impute mean late_rate for passed students missing it.
mean_pass_late_rate_custom = round(student_df_custom.loc[(student_df_custom['final_result'] == 2) & (student_df_custom['late_rate'].notna()), 'late_rate'].mean(), 2)
student_df_custom.loc[(student_df_custom['final_result'] == 2) & (student_df_custom['late_rate'].isna()), 'late_rate'] = mean_pass_late_rate_custom

### Fail Rate ###
# Impute mean fail_rate for passed students missing it.
mean_pass_fail_rate_custom = round(student_df_custom.loc[(student_df_custom['final_result'] == 2) & (student_df_custom['fail_rate'].notna()), 'fail_rate'].mean(), 2)
student_df_custom.loc[(student_df_custom['final_result'] == 2) & (student_df_custom['fail_rate'].isna()), 'fail_rate'] = mean_pass_fail_rate_custom

# Fill remaining missing late_rate and fail_rate with 0.
student_df_custom['late_rate'] = student_df_custom['late_rate'].fillna(1)
student_df_custom['fail_rate'] = student_df_custom['fail_rate'].fillna(1)

## IMD Band ##
imd_mapping = {
    '0-10%': 0,
    '10-20': 1,
    '20-30%': 2,
    '30-40%': 3,
    '40-50%': 4,
    '50-60%': 5,
    '60-70%': 6,
    '70-80%': 7,
    '80-90%': 8,
    '90-100%': 9
}
student_df_custom['imd_band'] = student_df_custom['imd_band'].map(imd_mapping)

# Prepare the data.
features = ['region', 'age_band', 'highest_education', 'gender', 'disability', 'final_result']
bayesian_imd_df = student_df_custom[features + ['imd_band']].copy()
bayesian_imd_df = bayesian_imd_df.dropna(subset=features) # Drop rows with missing values in the features

# Separate rows with known and missing imd_band.
train_imd_df = bayesian_imd_df[bayesian_imd_df['imd_band'].notnull()]
predict_imd_df = bayesian_imd_df[bayesian_imd_df['imd_band'].isnull()]

# Define columns to encode.
onehot_imd_cols = ['region', 'gender']
ordinal_imd_cols = ['age_band', 'highest_education', 'disability', 'final_result']

# Set up column transformer.
preprocessor = ColumnTransformer(transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), onehot_imd_cols),
        ('ordinal', OrdinalEncoder(), ordinal_imd_cols)
    ])

# Fit on training data and transform both train and predict sets.
X_train = preprocessor.fit_transform(train_imd_df[features])
X_predict = preprocessor.transform(predict_imd_df[features])
y_train = train_imd_df['imd_band'].astype(float)

# Initiate and fit the bayesian model.
bayesian_imd_model = BayesianRidge()
bayesian_imd_model.fit(X_train, y_train)

predicted_imd = bayesian_imd_model.predict(X_predict).round().clip(0, 9).astype(int) # Predict missing imd values.
student_df_custom.loc[student_df_custom['imd_band'].isnull(), 'imd_band'] = predicted_imd # Impute the imd predictions into the custom dataset.

## Date Unregistration ##
student_df_custom.drop(columns='date_unregistration', inplace=True)

In [99]:
student_df_custom.to_csv('student_df_full.csv', index=False)